In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
  .appName('Spotify Top Songs') \
  .config('spark.jars.packages', 'com.google.cloud.spark:spark-3.3-bigquery:0.37.0') \
  .getOrCreate()

24/04/01 14:38:21 WARN Utils: Your hostname, codespaces-64177b resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/04/01 14:38:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/python/3.10.13/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
com.google.cloud.spark#spark-3.3-bigquery added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1dcc3418-442c-49ef-b085-840bffe883ac;1.0
	confs: [default]
	found com.google.cloud.spark#spark-3.3-bigquery;0.37.0 in central
	found com.google.cloud.spark#spark-bigquery-dsv2-common;0.37.0 in central
	found com.google.cloud.spark#spark-bigquery-connector-common;0.37.0 in central
	found com.google.cloud.spark#bigquery-connector-common;0.37.0 in central
	found com.google.api.grpc#grpc-google-cloud-bigquerystorage-v1;3.3.1 in central
	found io.grpc#grpc-api;1.62.2 in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found com.google.errorprone#error_prone_annotations;2.23.0 in central
	found io.grpc#grpc-stub;1.62.2 in central
	found io.grpc#grpc-protobuf;1.62.2 in central
	found com.google.api.grpc#proto-google-common-protos;2.3

In [3]:
df = spark.read \
        .format("bigquery") \
        .option("table", "dezoomcampproject.spotify.top_songs_partitoned_clustered") \
        .load()

In [4]:
df.printSchema()


root
 |-- spotify_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- daily_rank: long (nullable = true)
 |-- daily_movement: long (nullable = true)
 |-- weekly_movement: long (nullable = true)
 |-- country: string (nullable = true)
 |-- snapshot_date: timestamp (nullable = true)
 |-- popularity: long (nullable = true)
 |-- is_explicit: boolean (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- album_name: string (nullable = true)
 |-- album_release_date: timestamp (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: long (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |

In [5]:
df.show()

24/04/01 14:39:03 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+----------+--------------+---------------+-------+-------------------+----------+-----------+-----------+--------------------+-------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------------+
|          spotify_id|                name|             artists|daily_rank|daily_movement|weekly_movement|country|      snapshot_date|popularity|is_explicit|duration_ms|          album_name| album_release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|__index_level_0__|
+--------------------+--------------------+--------------------+----------+--------------+---------------+-------+-------------------+----------+-----------+-----------+--------------------+-------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+---

In [6]:
#Convert the column 'snapshot_date' from datetime to date.
df= df.withColumn("snapshot_date", F.to_date(F.col("snapshot_date")))

In [7]:
#Convert the column 'album_release_date' from datetime to date.
df= df.withColumn("album_release_date", F.to_date(F.col("album_release_date")))


In [8]:
# Substitute null values in the 'country' column with 'Global'
df = df.withColumn("country", F.when(F.col("country").isNull(), "Global").otherwise(F.col("country")))

In [9]:
global_df = df.filter(df["country"] == "Global")


In [10]:
# Show the filtered DataFrame
global_df.show()

+--------------------+--------------------+--------------------+----------+--------------+---------------+-------+-------------+----------+-----------+-----------+--------------------+------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------------+
|          spotify_id|                name|             artists|daily_rank|daily_movement|weekly_movement|country|snapshot_date|popularity|is_explicit|duration_ms|          album_name|album_release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|__index_level_0__|
+--------------------+--------------------+--------------------+----------+--------------+---------------+-------+-------------+----------+-----------+-----------+--------------------+------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------

In [11]:
# Write transformed data back to BigQuery
df.write \
    .format("bigquery") \
    .option("temporaryGcsBucket", "dezoomcampproject-terra-bucket") \
    .option("table", "dezoomcampproject.spotify.top_songs_partitoned_clustered") \
    .mode("overwrite") \
    .save()